In [ ]:
import os
import torch
print("PyTorch has version {}".format(torch.__version__))

PyTorch has version 2.4.0+cu121


In [ ]:
if 'IS_GRADESCOPE_ENV' not in os.environ:
  !pip install torch==2.4.0

In [ ]:
# Install torch geometric
if 'IS_GRADESCOPE_ENV' not in os.environ:
  torch_version = str(torch.__version__)
  scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
  !pip install torch-scatter -f $scatter_src
  !pip install torch-sparse -f $sparse_src
  !pip install torch-geometric
  !pip install ogb

Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 50.5 MB/s eta 0:00:00
Looking in links: https://pytorch-geometric.com/whl/torch-2.4.0+cu121.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 3.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, SAGEConv, GraphConv
from torch.nn import Linear
from torch_geometric.datasets import TUDataset

In [ ]:
dataset = TUDataset(root='/dataset', name='PROTEINS')

In [ ]:
dataset

PROTEINS(1113)

In [ ]:
data = dataset[0]
data.edge_index


tensor([[ 0,  0,  0,  1,  1,  1,  2,  2,  2,  2,  3,  3,  3,  3,  3,  4,  4,  4,
          4,  4,  5,  5,  5,  5,  6,  6,  6,  6,  7,  7,  7,  7,  7,  8,  8,  8,
          8,  9,  9,  9, 10, 10, 10, 11, 11, 11, 11, 12, 12, 12, 12, 12, 13, 13,
         13, 13, 14, 14, 14, 15, 15, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18,
         18, 18, 18, 19, 19, 19, 20, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23,
         23, 24, 24, 24, 24, 24, 25, 25, 25, 25, 26, 26, 26, 27, 27, 27, 28, 28,
         28, 28, 29, 29, 29, 30, 30, 30, 31, 31, 31, 32, 32, 32, 33, 33, 33, 34,
         34, 34, 35, 35, 35, 35, 36, 36, 36, 37, 37, 37],
        [ 1,  2,  3,  0,  2,  3,  0,  1,  3,  4,  0,  1,  2,  4,  5,  2,  3,  5,
          6,  7,  3,  4,  6,  7,  4,  5,  7,  8,  4,  5,  6,  8,  9,  6,  7,  9,
         10,  7,  8, 10,  8,  9, 11, 10, 12, 27, 28, 11, 13, 14, 27, 28, 12, 14,
         15, 28, 12, 13, 15, 13, 14, 16, 17, 18, 15, 17, 18, 15, 16, 18, 15, 16,
         17, 19, 37, 18, 30, 37, 33, 34, 35, 36, 22

In [ ]:
print()
print(f'Dataset: {dataset}:')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')


Dataset: PROTEINS(1113):
Number of graphs: 1113
Number of features: 3
Number of classes: 2


In [ ]:
dataset = dataset.shuffle()

In [ ]:
train_dataset = dataset[:890]
test_dataset = dataset[890:]

In [ ]:
train_dataset, test_dataset

(PROTEINS(890), PROTEINS(223))

In [ ]:
from torch_geometric.loader import DataLoader
from torch_geometric.nn import  global_mean_pool, global_max_pool, global_add_pool

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
for step, data in enumerate(train_loader):
    print(f'Step {step + 1}:')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()

Step 1:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 12398], x=[3390, 3], y=[64], batch=[3390], ptr=[65])

Step 2:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 8888], x=[2388, 3], y=[64], batch=[2388], ptr=[65])

Step 3:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 8634], x=[2293, 3], y=[64], batch=[2293], ptr=[65])

Step 4:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 10878], x=[3004, 3], y=[64], batch=[3004], ptr=[65])

Step 5:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 9834], x=[2548, 3], y=[64], batch=[2548], ptr=[65])

Step 6:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 8492], x=[2215, 3], y=[64], batch=[2215], ptr=[65])

Step 7:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 10324], x=[2718, 3], y=[64], batch=[2718], ptr=[65])

Step 8:
Number of graphs in the current batch: 64
DataBatch(edge_index=[2, 7522], x=[2037, 3], y=[64]

In [ ]:
class GNN_Model(torch.nn.Module):
  def __init__(self, hidden_dim):
    super().__init__()
    self.conv1 = GraphConv(dataset.num_node_features, hidden_dim)
    self.conv2 = GraphConv(hidden_dim, hidden_dim)
    self.conv3 = GraphConv(hidden_dim, hidden_dim)
    self.linear = Linear(hidden_dim, dataset.num_classes)

  def forward(self, x, edge_list, batch):
    output = self.conv1(x, edge_list)
    output = F.relu(output)

    output = self. conv2(output, edge_list)
    output = F.relu(output)

    output = self. conv3(output, edge_list)

    output = global_max_pool(output, batch)
    print(output[0])
    output = F.dropout(output, p=0.5, training=self.training)
    output = self.linear(output)
    print("--------------")
    print(output)
    return output

model = GNN_Model(hidden_dim=64)
print(model)

GNN_Model(
  (conv1): GraphConv(3, 64)
  (conv2): GraphConv(64, 64)
  (conv3): GraphConv(64, 64)
  (linear): Linear(in_features=64, out_features=2, bias=True)
)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.05)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:

         out = model(data.x, data.edge_index, data.batch)
         loss = criterion(out, data.y)
         loss.backward()
         optimizer.step()
         optimizer.zero_grad()

def test(loader):
     model.eval()

     correct = 0
     for data in loader:
         out = model(data.x, data.edge_index, data.batch)
         pred = out.argmax(dim=1)
         correct += int((pred == data.y).sum())
     return correct / len(loader.dataset)


In [ ]:

for epoch in range(1, 171):
    train()
    train_acc = test(train_loader)
    test_acc = test(test_loader)
    print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')

Epoch: 001, Train Acc: 0.6753, Test Acc: 0.7354
Epoch: 002, Train Acc: 0.5921, Test Acc: 0.6188
Epoch: 003, Train Acc: 0.5921, Test Acc: 0.6143
Epoch: 004, Train Acc: 0.6730, Test Acc: 0.7085
Epoch: 005, Train Acc: 0.6618, Test Acc: 0.6951
Epoch: 006, Train Acc: 0.6876, Test Acc: 0.7265
Epoch: 007, Train Acc: 0.6618, Test Acc: 0.6771
Epoch: 008, Train Acc: 0.6573, Test Acc: 0.6816
Epoch: 009, Train Acc: 0.7416, Test Acc: 0.7623
Epoch: 010, Train Acc: 0.7427, Test Acc: 0.7713
Epoch: 011, Train Acc: 0.7427, Test Acc: 0.7668
Epoch: 012, Train Acc: 0.7382, Test Acc: 0.7399
Epoch: 013, Train Acc: 0.7416, Test Acc: 0.7668
Epoch: 014, Train Acc: 0.7382, Test Acc: 0.7399
Epoch: 015, Train Acc: 0.6371, Test Acc: 0.6682
Epoch: 016, Train Acc: 0.7157, Test Acc: 0.7085
Epoch: 017, Train Acc: 0.6876, Test Acc: 0.7130
Epoch: 018, Train Acc: 0.6697, Test Acc: 0.6726
Epoch: 019, Train Acc: 0.6427, Test Acc: 0.6771
Epoch: 020, Train Acc: 0.7236, Test Acc: 0.7175
Epoch: 021, Train Acc: 0.7157, Test Acc:

In [ ]:
for i in train_loader:
  out = model(i.x, i.edge_index, i.batch)

  #print(out.argmax(dim=-1))
  print(i.y)
  break

tensor([ 0.9818,  0.9971,  6.0202,  5.3406,  1.2014,  2.3558,  0.0406,  0.0934,
         1.4875,  0.6573, -1.4561,  1.9957,  1.1816,  1.3647,  3.2256, -1.3312,
        -0.0300, -0.7867, -1.0180, -0.7296,  1.4500,  1.3470,  4.4886,  2.5111,
         3.9514,  0.7565, -1.5890,  1.2457,  3.6841,  0.3431,  3.4732,  2.8634,
         1.9765, -0.0609,  2.0430, -1.3839, -0.7241, -0.2838,  3.0717,  0.3545,
        -0.8081, -3.0104,  0.1873, -0.6753, -0.8333, -1.4015,  2.1929,  0.5337,
        -0.4195, -0.6869,  0.8399,  5.2491,  4.5713,  0.8523,  1.9758,  0.4263,
         2.7447,  0.2810, -0.0907,  3.7960, -1.2919,  0.6655,  2.3635,  0.3167],
       grad_fn=<SelectBackward0>)
--------------
tensor([[-9.9772e-01,  2.6391e-01],
        [-2.9706e+00,  3.2323e+00],
        [-2.5653e+00,  3.7124e+00],
        [-1.9242e+00,  2.6656e-01],
        [ 7.6880e-01,  4.3681e+00],
        [-3.8006e-01,  1.2871e-01],
        [-1.8327e+00,  1.7827e+00],
        [ 1.5447e+00,  2.9412e+00],
        [-4.2100e-01, 